In [ ]:
import cv2
import numpy as np
from typing import List, Tuple, Optional

def Rotate_Image(input_image: np.ndarray, angle_degrees: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    angle_degrees = np.clip(angle_degrees, -30, 30)
    
    height, width = input_image.shape[:2]
    center = (width / 2, height / 2)
    
    rotation_matrix = cv2.getRotationMatrix2D(center, angle_degrees, scale=1.0)
    
    rotated_image = cv2.warpAffine(
        input_image,
        rotation_matrix,
        (width, height),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )
    
    return rotated_image

def Flip_Horizontal(input_image: np.ndarray) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    return cv2.flip(input_image, 1)

def Flip_Vertical(input_image: np.ndarray) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    return cv2.flip(input_image, 0)

def Random_Crop(input_image: np.ndarray, crop_height: int, crop_width: int) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    height, width = input_image.shape[:2]
    
    if crop_height > height or crop_width > width:
        return input_image
    
    max_y = height - crop_height
    max_x = width - crop_width
    
    y_start = np.random.randint(0, max_y + 1) if max_y > 0 else 0
    x_start = np.random.randint(0, max_x + 1) if max_x > 0 else 0
    
    cropped_image = input_image[y_start:y_start + crop_height, x_start:x_start + crop_width]
    
    return cropped_image

def Resize_Image(input_image: np.ndarray, target_height: int, target_width: int) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    resized_image = cv2.resize(
        input_image,
        (target_width, target_height),
        interpolation=cv2.INTER_LINEAR
    )
    
    return resized_image

def Mosaic_Augmentation(image_batch_list: List[np.ndarray], 
                        target_height: int, target_width: int) -> np.ndarray:
    if len(image_batch_list) != 4:
        raise ValueError(f"Mosaic requires exactly 4 images, got {len(image_batch_list)}")
    
    for i, img in enumerate(image_batch_list):
        if img is None or img.size == 0:
            raise ValueError(f"Image at index {i} is empty or None")
    
    half_height = target_height // 2
    half_width = target_width // 2
    
    resized_images = []
    for img in image_batch_list:
        resized = Resize_Image(img, half_height, half_width)
        resized_images.append(resized)
    
    if len(resized_images[0].shape) == 3:
        channels = resized_images[0].shape[2]
        mosaic = np.zeros((target_height, target_width, channels), dtype=resized_images[0].dtype)
    else:
        mosaic = np.zeros((target_height, target_width), dtype=resized_images[0].dtype)
    
    mosaic[0:half_height, 0:half_width] = resized_images[0]
    mosaic[0:half_height, half_width:target_width] = resized_images[1]
    mosaic[half_height:target_height, 0:half_width] = resized_images[2]
    mosaic[half_height:target_height, half_width:target_width] = resized_images[3]
    
    return mosaic

def Mixup_Augmentation(image_1: np.ndarray, image_2: np.ndarray, 
                       mixup_alpha: float) -> np.ndarray:
    if image_1 is None or image_1.size == 0:
        raise ValueError("image_1 is empty or None")
    if image_2 is None or image_2.size == 0:
        raise ValueError("image_2 is empty or None")
    
    mixup_alpha = np.clip(mixup_alpha, 0.2, 0.4)
    
    if image_1.shape != image_2.shape:
        target_height, target_width = image_1.shape[:2]
        image_2 = cv2.resize(image_2, (target_width, target_height), interpolation=cv2.INTER_LINEAR)
    
    img1_float = image_1.astype(np.float32)
    img2_float = image_2.astype(np.float32)
    
    blended = mixup_alpha * img1_float + (1 - mixup_alpha) * img2_float
    blended = np.clip(blended, 0, 255)
    
    return blended.astype(image_1.dtype)

def Ripple_Distortion(input_image: np.ndarray, amplitude: float, frequency: float) -> Optional[np.ndarray]:
    if input_image is None: return None
    h, w = input_image.shape[:2]
    
    frequency = max(frequency, 0.1) 
    wavelength = w / frequency

    x, y = np.meshgrid(np.arange(w), np.arange(h))
    
    cx, cy = w // 2, h // 2
    
    distance = np.sqrt((x - cx)**2 + (y - cy)**2)
    
    displacement = amplitude * np.sin(distance / wavelength * 2 * np.pi)
    
    map_x = (x + displacement).astype(np.float32)
    map_y = (y + displacement).astype(np.float32)
    
    distorted = cv2.remap(
        input_image, 
        map_x, 
        map_y, 
        interpolation=cv2.INTER_LINEAR, 
        borderMode=cv2.BORDER_REFLECT
    )
    
    return distorted

def Wave_Distortion(input_image: np.ndarray, amplitude: float, frequency: float, 
                   wave_direction: str = 'horizontal') -> Optional[np.ndarray]:
    if input_image is None: return None
    h, w = input_image.shape[:2]
    
    frequency = max(frequency, 0.1)
    wavelength = w / frequency

    x, y = np.meshgrid(np.arange(w), np.arange(h))

    if wave_direction == 'horizontal':
        map_x = x + amplitude * np.sin(2 * np.pi * y / wavelength)
        map_y = y
    else:
        map_x = x
        map_y = y + amplitude * np.sin(2 * np.pi * x / wavelength)

    map_x = map_x.astype(np.float32)
    map_y = map_y.astype(np.float32)

    distorted = cv2.remap(
        input_image, 
        map_x, 
        map_y, 
        interpolation=cv2.INTER_LINEAR, 
        borderMode=cv2.BORDER_REFLECT
    )
    return distorted

def Bubbles_Overlay(input_image: np.ndarray, bubble_count: int = 10) -> Optional[np.ndarray]:
    if input_image is None: return None
    
    result = input_image.copy()
    h, w = result.shape[:2]
    
    for _ in range(bubble_count):
        cx = np.random.randint(0, w)
        cy = np.random.randint(0, h)
        radius = np.random.randint(5, 20)
        
        cv2.circle(result, (cx, cy), radius, (255, 255, 255), 1)
        cv2.circle(result, (cx - radius//3, cy - radius//3), 1, (255, 255, 255), 1)
        
    return result
    
def Caustic_Overlay(input_image: np.ndarray, blend_strength: float = 0.2) -> Optional[np.ndarray]:
    if input_image is None: return None
    
    img_float = input_image.astype(np.float32) / 255.0
    h, w = img_float.shape[:2]

    scale = int(min(h, w) * 0.03) 
    scale = max(scale, 5)     
    
    noise = np.random.rand(h, w).astype(np.float32)
    noise = cv2.GaussianBlur(noise, (0, 0), scale)
    
    gx = cv2.Sobel(noise, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(noise, cv2.CV_32F, 0, 1, ksize=3)
    
    caustic_pattern = np.sqrt(gx**2 + gy**2)
    
    caustic_pattern = cv2.normalize(caustic_pattern, None, 0, 1, cv2.NORM_MINMAX)
    caustic_pattern = np.power(caustic_pattern, 3) 
    
    if len(input_image.shape) == 3:
        caustic_pattern = np.repeat(caustic_pattern[:, :, np.newaxis], 3, axis=2)

    result = img_float + (caustic_pattern * blend_strength)
    
    result = np.clip(result, 0, 1)
    return (result * 255).astype(np.uint8)

def Debris_Overlay(input_image: np.ndarray, debris_count: int = 50) -> Optional[np.ndarray]:
    if input_image is None: return None
        
    h, w = input_image.shape[:2]
    debris_layer = np.zeros((h, w, 3), dtype=np.uint8)
    
    for _ in range(debris_count):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        
        color_intensity = np.random.randint(100, 255)
        color = (color_intensity, color_intensity, color_intensity)
        
        size = np.random.randint(1, 4)
        cv2.circle(debris_layer, (x, y), size, color, -1)

    debris_layer = cv2.GaussianBlur(debris_layer, (3, 3), 0)
    
    result = cv2.add(input_image, debris_layer)
    
    return result

def Color_Jitter_Brightness(input_image: np.ndarray, brightness_factor: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    brightness_factor = np.clip(brightness_factor, 0.8, 1.2)
    
    brightened = input_image.astype(np.float32) * brightness_factor
    brightened = np.clip(brightened, 0, 255)
    
    return brightened.astype(np.uint8)

def Color_Jitter_Contrast(input_image: np.ndarray, contrast_factor: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    contrast_factor = np.clip(contrast_factor, 0.8, 1.2)
    
    img_float = input_image.astype(np.float32)
    mean = np.mean(img_float, axis=(0, 1), keepdims=True)
    
    contrasted = (img_float - mean) * contrast_factor + mean
    contrasted = np.clip(contrasted, 0, 255)
    
    return contrasted.astype(np.uint8)

def Color_Jitter_Saturation(input_image: np.ndarray, saturation_factor: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    if len(input_image.shape) != 3 or input_image.shape[2] != 3:
        raise ValueError("Input image must be a 3-channel BGR image")
    
    saturation_factor = np.clip(saturation_factor, 0.5, 2.0)
    
    hsv = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV).astype(np.float32)
    
    hsv[:, :, 1] = hsv[:, :, 1] * saturation_factor
    hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
    
    hsv = hsv.astype(np.uint8)
    saturated = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    return saturated

def Color_Jitter_Hue(input_image: np.ndarray, hue_shift_value: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    if len(input_image.shape) != 3 or input_image.shape[2] != 3:
        raise ValueError("Input image must be a 3-channel BGR image")
    
    hue_shift_value = np.clip(hue_shift_value, -10, 10)
    
    hsv = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV).astype(np.float32)
    
    hsv[:, :, 0] = (hsv[:, :, 0] + hue_shift_value) % 180
    
    hsv = hsv.astype(np.uint8)
    hue_shifted = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    return hue_shifted

def Attenuate_Channel(input_image: np.ndarray, channel_index: int, 
                      attenuation_factor: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    if len(input_image.shape) != 3 or input_image.shape[2] != 3:
        raise ValueError("Input image must be a 3-channel BGR image")
    
    if channel_index not in [0, 1, 2]:
        raise ValueError("channel_index must be 0 (Blue), 1 (Green), or 2 (Red)")
    
    attenuation_factor = np.clip(attenuation_factor, 0.6, 0.9)
    
    attenuated = input_image.astype(np.float32).copy()
    attenuated[:, :, channel_index] = attenuated[:, :, channel_index] * attenuation_factor
    attenuated = np.clip(attenuated, 0, 255)
    
    return attenuated.astype(np.uint8)

def Adjust_Gamma(input_image: np.ndarray, gamma_value: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    gamma_value = np.clip(gamma_value, 0.8, 1.2)
    
    inv_gamma = 1.0 / gamma_value
    lut = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype(np.uint8)
    
    gamma_corrected = cv2.LUT(input_image, lut)
    
    return gamma_corrected

def Add_Haze(input_image: np.ndarray, haze_intensity: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    haze_intensity = np.clip(haze_intensity, 0.1, 0.3)
    
    img_float = input_image.astype(np.float32)
    white = np.ones_like(img_float) * 255
    
    hazed = img_float * (1 - haze_intensity) + white * haze_intensity
    hazed = np.clip(hazed, 0, 255)
    
    return hazed.astype(np.uint8)

def Add_Fog(input_image: np.ndarray, scattering_coefficient: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    scattering_coefficient = np.clip(scattering_coefficient, 0.6, 2.5)
    
    height, width = input_image.shape[:2]
    
    depth_map = np.linspace(0, 1, width, dtype=np.float32)
    depth_map = np.tile(depth_map, (height, 1))
    
    transmission = np.exp(-scattering_coefficient * depth_map)
    
    if len(input_image.shape) == 3:
        transmission = transmission[:, :, np.newaxis]
    
    atmospheric_light = 200
    
    img_float = input_image.astype(np.float32)
    fogged = img_float * transmission + atmospheric_light * (1 - transmission)
    fogged = np.clip(fogged, 0, 255)
    
    return fogged.astype(np.uint8)

def Underwater_Turbidity(input_image: np.ndarray, red_attn: float, 
                        green_attn: float, blue_attn: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    if len(input_image.shape) != 3 or input_image.shape[2] != 3:
        raise ValueError("Input image must be a 3-channel BGR image")
    
    red_attn = np.clip(red_attn, 0.4, 0.7)
    green_attn = np.clip(green_attn, 0.7, 0.9)
    blue_attn = np.clip(blue_attn, 0.6, 0.9)
    
    turbid = input_image.astype(np.float32).copy()
    
    turbid[:, :, 0] = turbid[:, :, 0] * blue_attn
    turbid[:, :, 1] = turbid[:, :, 1] * green_attn
    turbid[:, :, 2] = turbid[:, :, 2] * red_attn
    
    turbid = np.clip(turbid, 0, 255)
    
    return turbid.astype(np.uint8)

def Scattering_Blur(input_image: np.ndarray, sigma_value: float) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    sigma_value = np.clip(sigma_value, 0.5, 2.0)
    
    kernel_size = int(2 * np.ceil(3 * sigma_value) + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1
    
    blurred = cv2.GaussianBlur(input_image, (kernel_size, kernel_size), sigmaX=sigma_value)
    
    return blurred

def Multi_Color_Space_Fusion(input_image: np.ndarray) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")

    img_hsv = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV).astype(np.float32)
    img_lab = cv2.cvtColor(input_image, cv2.COLOR_BGR2LAB).astype(np.float32)

    v_channel = img_hsv[:, :, 2]
    l_channel = img_lab[:, :, 0]

    fused_v = (0.5 * v_channel) + (0.5 * l_channel)
    
    img_hsv[:, :, 2] = fused_v
    img_hsv = np.clip(img_hsv, 0, 255).astype(np.uint8)

    fused_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
    
    return fused_image

def Channel_wise_Multi_scale(input_image: np.ndarray) -> np.ndarray:
    if input_image is None or input_image.size == 0:
        raise ValueError("Input image is empty or None")
    
    img_float = input_image.astype(np.float32)
    
    scales = [1.0, 2.0, 4.0]
    weights = [0.5, 0.3, 0.2] 
    
    detail_accumulator = np.zeros_like(img_float)
    
    for sigma, weight in zip(scales, weights):
        ksize = int(2 * np.ceil(3 * sigma) + 1)
        blurred = cv2.GaussianBlur(img_float, (ksize, ksize), sigma)
        
        detail = img_float - blurred
        
        detail_accumulator += detail * weight
        
    sharpened = img_float + detail_accumulator
    
    sharpened = np.clip(sharpened, 0, 255)
    return sharpened.astype(np.uint8)